In [1]:
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

def prepare_dataframe(folder_dir):
    res = []
    for root, dirs, files in os.walk(folder_dir, topdown=True):
        if len(files) > 0:
            res.extend(list(zip([root]*len(files), files)))
    
    df = pd.DataFrame(res, columns=['Path', 'File_Name'])
    df = df[df['File_Name'] != 'Thumbs.db']
    #df['ClientId'] = df.Path.apply(lambda x: int(x.split("\\")[-1]))
    #df = df[df['ClientId'] < 10000]

    df['Full_Path'] = df["Path"] + '\\' + df["File_Name"]
    df['Cat'] = df.File_Name.apply(lambda x: x.split(".")[0].split("_")[-1])

    df = df[df['Cat'].map(df['Cat'].value_counts()) > 150]

    return df

In [2]:
from transferlearning import ConvBaseSearch

pretrained_models = ["vgg16", "vgg19"] #Models that should be tested
#folder_dir = 'C:\\Users\\Michael\\Feels Like Home\\Francisco Cruz - Fotos_DL\\011_Fotos\\'
folder_dir = 'C:\\Users\\Michael\\Desktop\\Master\\Deep Learning\\Project\\011_Fotos\\'

df = prepare_dataframe(folder_dir)
target_col = "Cat"
path_col = "Full_Path"

Using TensorFlow backend.


In [3]:
# Initialize ConvBaseSearch and pass the model names you want to test
cbs = ConvBaseSearch(pretrained_models)

In [4]:
# Fit the data to the pretrained models. Only use sample_size proportion of whole dataset.
cbs.fit(df, path_col, target_col, sample_size=0.02, data_augmentation=True)

Found 178 validated image filenames belonging to 12 classes.
Found 45 validated image filenames belonging to 12 classes.
Fitting  vgg16
Epoch 1/1
2/2 [==============================] - 57s 29s/step - loss: 2.8959 - accuracy: 0.1026
Score on test set:  0.2222222238779068
Fitting  vgg19
Epoch 1/1
2/2 [==============================] - 60s 30s/step - loss: 3.2224 - accuracy: 0.0200
Score on test set:  0.06666667014360428


In [5]:
# Get the best score 
cbs.best_score

0.2222222238779068

In [6]:
# Get the best model
best_base = cbs.best_base
print("Name:", best_base.name)
print("Keras model:", best_base.model)

Name: vgg16
Keras model: <keras.engine.training.Model object at 0x000002C074E10F88>
